# Project: '2022 Fall Student Project: Deep Supervised Learning for MRI Reconstruction' 
This project is authored by: Weijie Gan and Yuyang Hu on July 14th, 2022
Project description: main.pdf
<br>In this project, I try to implement the pipeline described in this project.

In [1]:
import numpy as np
import math
import h5py

In [2]:
import tensorflow as tf

In [2]:
filename = "dataset.hdf5"
data = h5py.File(filename,'r')

In this project, the linear model is:
\begin{equation}
\textit{\textbf{y}} = \textit{\textbf{PFx}} + \textit{e}
\end{equation}
$\textit{\textbf{x}}\in\mathbb{C}^n$ is an image
<br>$\textit{\textbf{x}}\in\mathbb{C}^n$ is a noisy measurement; this is a size-N typle of complex numbers
<br>$\textit{\textbf{F}}\in\mathbb{C}^{n\times n}$ is the Fourier transform; this is a NxN array complex numbers
<br>$\textit{\textbf{P}}\in\mathbb{C}^{n\times n}$ is a sampling operator; this is a NxN array complex numbers
<br>$\textit{\textbf{e}}\in\mathbb{C}^{n\times n}$ is a noise vector; this is a NxN array complex numbers

Our loss function is:
\begin{equation}
\frac{1}{N}\sum_{i}^N \big\|f_\theta(\^{\textit{\textbf{x}}}_i)-\textit{\textbf{x}}_i\big\|^2_2
\end{equation}
$f_\theta$ is a Convolutional Neural Network. It maps $\bigl\{\^{\textit{\textbf{x}}}_i=\textit{\textbf{F}}^{-1}\textit{\textbf{y}}_i\bigl\}^N_i$ to their desired ground truth $\bigl\{\textit{\textbf{x}}_i\bigl\}^N_i$
<br>$\textit{\textbf{F}}^{-1}$ is inverse Fourier transform
<br>$N$ is total number of training samples


\begin{equation}
\widehat{x}_i
\end{equation}

In [3]:
with h5py.File('dataset.hdf5', 'r') as hdf:
    keys = list(hdf.keys())
    trnOrg = hdf.get('trnOrg')
    ground_truth = np.array(trnOrg)
    trnMask = hdf.get('trnMask')
    scaling_operator = np.array(trnOrg)

In [4]:
print(keys)
print(ground_truth.shape)
print(scaling_operator.shape)

['trnCsm', 'trnMask', 'trnOrg', 'tstCsm', 'tstMask', 'tstOrg']
(360, 256, 232)
(360, 256, 232)


In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 11, 11, 32)        9248      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 5, 5, 32)         0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 800)               0         
                                                                 
 dense_4 (Dense)             (None, 128)              

model.fit()